In [ ]:
# mount to drive
from google.colab import drive, auth
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qU transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time
import transformers
import torch

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
print(auth.authenticate_user(project_id='mlhc-spring-2025'))

None


In [ ]:
!pip install --upgrade -q gspread

In [ ]:
import gspread
from google.auth import default
creds, _ = default()

In [ ]:
gc = gspread.authorize(creds)

In [ ]:
path_to_worksheet = 'LLM Prompt Evaluation'
worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]  # Set the first row as column names
prompt_df.drop(prompt_df.index[0], inplace=True)  # Drop the first row since it's now the header
prompt_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes
1,John Snow Labs 2.78B gguf,Sasha,1,1,You are a vascular surgeon providing help to a...,B. Left carotid endarterectomy with shunt plac...,No,
2,John Snow Labs 2.78B gguf,Sasha,2,1,You are a vascular surgeon providing help to a...,You are a vascular surgeon guiding a medical s...,Yes,
3,John Snow Labs 2.78B gguf,Sasha,3,1,You are a vascular surgeon providing help to a...,You are a vascular surgeon guiding a medical s...,Yes,
4,John Snow Labs 2.78B gguf,Sasha,4,1,You are a vascular surgeon providing help to a...,You are a vascular surgeon guiding a medical s...,No,
5,John Snow Labs 2.78B gguf,Sasha,5,1,You are a vascular surgeon providing help to a...,You are a vascular surgeon guiding a medical s...,No,
...,...,...,...,...,...,...,...,...
503,GPT2,Sasha,9,3,You are a vascular surgeon guiding a medical s...,,,
504,GPT2,Sasha,10,3,You are a vascular surgeon guiding a medical s...,,,
505,GPT2,Sasha,11,3,You are a vascular surgeon guiding a medical s...,,,
506,GPT2,Sasha,12,3,You are a vascular surgeon guiding a medical s...,,,


**MELT-TinyLlama**

In [ ]:
melt_tinyLlama_df = prompt_df[prompt_df['LLM Model'] == 'IBI-CAAI_-_MELT-TinyLlama']
melt_tinyLlama_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes
196,IBI-CAAI_-_MELT-TinyLlama,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,
197,IBI-CAAI_-_MELT-TinyLlama,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,
198,IBI-CAAI_-_MELT-TinyLlama,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,
199,IBI-CAAI_-_MELT-TinyLlama,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,
200,IBI-CAAI_-_MELT-TinyLlama,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,
201,IBI-CAAI_-_MELT-TinyLlama,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,
202,IBI-CAAI_-_MELT-TinyLlama,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,
203,IBI-CAAI_-_MELT-TinyLlama,Sasha,8,1,You are a vascular surgeon providing help to a...,,,
204,IBI-CAAI_-_MELT-TinyLlama,Sasha,9,1,You are a vascular surgeon providing help to a...,,,
205,IBI-CAAI_-_MELT-TinyLlama,Sasha,10,1,You are a vascular surgeon providing help to a...,,,


In [ ]:
model_id = "IBI-CAAI/MELT-TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer and pipeline
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Format the prompt per model expectations
query = "How does photosynthesis work?"
prompt = f"### User:\n{query}\n\n### Assistant:\n"

# Run generation
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

print(outputs[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Device set to use cuda:0


### User:
How does photosynthesis work?

### Assistant:
Photosynthesis is the process by which plants, algae, and some bacteria convert sunlight into chemical energy (energy in the form of ATP and NADPH) through the conversion of carbon dioxide and water into glucose and oxygen.


In [ ]:
id_response = {}
for index, row in melt_tinyLlama_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.4, top_k=50, top_p=0.95)
  print(index, id_response[index])

196 [{'generated_text': [{'role': 'user', 'content': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid end

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


203 [{'generated_text': [{'role': 'user', 'content': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n8. A 56-year-old man with chronic rate-controlled atrial fibrillation stopped coumadin prior to a colonoscopy 1 week ago. He is otherwise healthy and takes no other medications. He presents with pain in his left calf after walking two miles in 90 degree temperatures. He is found to have both absent pulses and doppler signals in both his dorsalis pedis and posterior tibial arteries in that leg. This is accompanied by pallor and pain. What is the most likely etiology?\nEmbolus as a result of his atrial fibrillation.\n\nEmbolus that originated from an aortic plaque.\n\nParadoxical embolus.\n\nThrombosi

In [ ]:
melt_tinyLlama_response_df = pd.DataFrame([ (x[0], x[1][0]["generated_text"][1]["content"]) for x in id_response.items()], columns=['id', 'sum'])
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(melt_tinyLlama_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
melt_tinyLlama_df = prompt_df[prompt_df['LLM Model'] == 'IBI-CAAI_-_MELT-TinyLlama']
melt_tinyLlama_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
195,IBI-CAAI_-_MELT-TinyLlama,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,B
196,IBI-CAAI_-_MELT-TinyLlama,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,C
197,IBI-CAAI_-_MELT-TinyLlama,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,B
198,IBI-CAAI_-_MELT-TinyLlama,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,B
199,IBI-CAAI_-_MELT-TinyLlama,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,B
200,IBI-CAAI_-_MELT-TinyLlama,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,B
201,IBI-CAAI_-_MELT-TinyLlama,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,B
202,IBI-CAAI_-_MELT-TinyLlama,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,B
203,IBI-CAAI_-_MELT-TinyLlama,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,B
204,IBI-CAAI_-_MELT-TinyLlama,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,B


In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = melt_tinyLlama_df.columns.tolist()
worksheet.append_row(cols)
for index, row in melt_tinyLlama_df.iterrows():
  worksheet.append_row(row.tolist())

**PHI-1.5**

In [ ]:
phi1_5_df = prompt_df[prompt_df['LLM Model'] == 'phi-1.5']
phi1_5_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
351,phi-1.5,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,NaN
352,phi-1.5,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,NaN
353,phi-1.5,Sasha,3,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
354,phi-1.5,Sasha,4,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
355,phi-1.5,Sasha,5,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
356,phi-1.5,Sasha,6,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
357,phi-1.5,Sasha,7,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
358,phi-1.5,Sasha,8,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
359,phi-1.5,Sasha,9,1,You are a vascular surgeon providing help to a...,"D""",,,NaN
360,phi-1.5,Sasha,10,1,You are a vascular surgeon providing help to a...,"D""",,,NaN


In [ ]:
from transformers import AutoModelForCausalLM

In [ ]:
model = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model)

# Prepare the pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

prompt = '''def print_prime(n):
    """
    Print all primes between 1 and n
    """'''

# Generate output using the pipeline
outputs = pipeline(prompt, max_new_tokens=200, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

# Print the generated text
print(outputs[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Device set to use cuda:0


def print_prime(n):
    """
    Print all primes between 1 and n
    """
    primes = [2]
    for num in range(3, n + 1, 2):
        is_prime = True
        for i in range(3, int(math.sqrt(num)) + 1, 2):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
    print(primes)

print_prime(20)
```

4. Write a Python program that creates a list of the squares of all even numbers between 1 and 20 using list comprehension.

```python
# Solution
even_squares = [x ** 2 for x in range(1, 21) if x % 2 == 0]
print(even_squares)
```

5. Write a Python program that creates a list of the absolute differences between consecutive elements in a list


In [ ]:
id_response = {}
for index, row in phi1_5_df.iterrows():
  prompt = row['Prompt + Question']
  id_response[index] = pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty = 1.2)
  print(index, id_response[index])

351 [{'generated_text': "You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Placement 

In [ ]:
answer_1 = next(iter(id_response.items()))
print(answer_1)
print(answer_1[0])
print(answer_1[1][0]['generated_text'])

(351, [{'generated_text': "You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Placemen

In [ ]:
phi1_5_response_df = pd.DataFrame([ (x[0], x[1][0]["generated_text"]) for x in id_response.items()], columns=['id', 'sum'])
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(phi1_5_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
phi1_5_df = prompt_df[prompt_df['LLM Model'] == 'phi-1.5']
phi1_5_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
351,phi-1.5,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,You are a vascular surgeon providing help to a...
352,phi-1.5,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,You are a vascular surgeon providing help to a...
353,phi-1.5,Sasha,3,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
354,phi-1.5,Sasha,4,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
355,phi-1.5,Sasha,5,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
356,phi-1.5,Sasha,6,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
357,phi-1.5,Sasha,7,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
358,phi-1.5,Sasha,8,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
359,phi-1.5,Sasha,9,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...
360,phi-1.5,Sasha,10,1,You are a vascular surgeon providing help to a...,"D""",,,You are a vascular surgeon providing help to a...


In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = phi1_5_df.columns.tolist()
worksheet.append_row(cols)
for index, row in phi1_5_df.iterrows():
  worksheet.append_row(row.tolist())

InvalidJSONError: Out of range float values are not JSON compliant

**GPT2 Base**

In [ ]:
gpt2_df = prompt_df[prompt_df['LLM Model'] == 'GPT2']
gpt2_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes
469,GPT2,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,
470,GPT2,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,
471,GPT2,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,
472,GPT2,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,
473,GPT2,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,
474,GPT2,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,
475,GPT2,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,
476,GPT2,Sasha,8,1,You are a vascular surgeon providing help to a...,,,
477,GPT2,Sasha,9,1,You are a vascular surgeon providing help to a...,,,
478,GPT2,Sasha,10,1,You are a vascular surgeon providing help to a...,,,


In [ ]:
model = "openai-community/gpt2"

# Use plain string, since this model is not chat-based
prompt = "What is a large language model?"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Create pipeline with proper precision and device
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # As noted in the model card
    device_map="auto",
)

# Generate output
outputs = pipeline(
    prompt,
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

print(outputs[0]["generated_text"])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is a large language model?

What is a small language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is a language model?

What is


In [ ]:
id_response = {}
for index, row in gpt2_df.iterrows():
  prompt = row['Prompt + Question']
  id_response[index] = pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty = 1.2)
  print(index, id_response[index])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


469 [{'generated_text': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Placement 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


470 [{'generated_text': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n2. What is the first muscle layer encountered in a carotid endarterectomy (i.e. the first muscle deep to skin)?\nA. Sternocleidomastoid\n\nB. Digastric\n\nC. Platysma\n\nD. Scalene\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\n(1) In this case, please describe what happened as described below: There was no injury from either side while performing an amputation procedure on my right forearm; however I am still feeling great pain after about 20 minutes when walking back down into hospital room where there were several people present with both hands touching each another du

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

502 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, D, or E). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n8. A 56-year-old man with chronic rate-controlled atrial fibrillation stopped coumadin prior to a colonoscopy 1 week ago. He is otherwise healthy and takes no other medications. He presents with pain in his left calf after walking two miles in 90 degree temperatures. He is found to have both absent pulses and doppler signals in both his dorsalis pedis and posterior tibial arteries in that leg. This is accompanied by pallor and pain. What is the most likely etiology?\nA. Embolus as a result of his atrial fibrillation.\n\nB. Embol

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


503 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n9. A 74 year old male with a history of metastatic pancreatic cancer, hypertension, prior HTN, hyperlipidemia presents to the ED with acute onset pain in his right leg. CT scan shows cutoff sign at the P3 segment just proximal to the right tibial trifurcation. In addition to pain, the patient endorses decreased sensation of his right lower leg. On exam his right leg is cool to touch and has absent pedal and popliteal doppler signals. What is the first step in management of this patient?\nA. Emergent open surgical embolectomy with b

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


504 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n\n10. A 55-year-old male presents with aching legs, swelling, and skin discoloration. Physical examination reveals dilated veins and hyperpigmentation. Which diagnostic modality is most appropriate initially?\nA Magnetic resonance imaging (MRI)\n\nB. Venous duplex ultrasound\n\nC. Arterial Doppler ultrasound\n\nD. Computed tomography (CT) scan\n\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\n: "I am on anticoagulant medication." The patient should also be asked about other me

In [ ]:
gpt2_response_df = pd.DataFrame([ (x[0], x[1][0]["generated_text"]) for x in id_response.items()], columns=['id', 'sum'])
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(gpt2_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
gpt2_df = prompt_df[prompt_df['LLM Model'] == 'GPT2']
gpt2_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
468,GPT2,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,You are a vascular surgeon providing help to a...
469,GPT2,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,You are a vascular surgeon providing help to a...
470,GPT2,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,You are a vascular surgeon providing help to a...
471,GPT2,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,You are a vascular surgeon providing help to a...
472,GPT2,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,You are a vascular surgeon providing help to a...
473,GPT2,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,You are a vascular surgeon providing help to a...
474,GPT2,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,You are a vascular surgeon providing help to a...
475,GPT2,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,You are a vascular surgeon providing help to a...
476,GPT2,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,You are a vascular surgeon providing help to a...
477,GPT2,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,You are a vascular surgeon providing help to a...


In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = gpt2_df.columns.tolist()
worksheet.append_row(cols)
for index, row in gpt2_df.iterrows():
  worksheet.append_row(row.tolist())

**GPT2 Fine Tuned**

In [ ]:
gpt2_pmc_df = prompt_df[prompt_df['LLM Model'] == 'GPT2-ft']
gpt2_pmc_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
429,GPT2-ft,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,NaN
430,GPT2-ft,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,NaN
431,GPT2-ft,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,NaN
432,GPT2-ft,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,NaN
433,GPT2-ft,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,NaN
434,GPT2-ft,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,NaN
435,GPT2-ft,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,NaN
436,GPT2-ft,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,NaN
437,GPT2-ft,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,NaN
438,GPT2-ft,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,NaN


In [ ]:
model = "manupande21/GPT2_PMC"

# Use plain string, since this model is not chat-based
prompt = "What is a large language model?"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Create pipeline with proper precision and device
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # As noted in the model card
    device_map="auto",
)

# Generate output
outputs = pipeline(
    prompt,
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

print(outputs[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is a large language model?   A large language model includes all words in a language, including the form and structure of the words, the types of words, the number of possible inflections, and the corresponding types of inflections. 
 What is the significance of the role of the phonological feature in the translation of a short form into a long form?   The phonological feature has the significance of the translation of a short form into a long


In [ ]:
id_response = {}
for index, row in gpt2_pmc_df.iterrows():
  prompt = row['Prompt + Question']
  id_response[index] = pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.2)
  print(index, id_response[index])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


429 [{'generated_text': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Placement 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


430 [{'generated_text': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n2. What is the first muscle layer encountered in a carotid endarterectomy (i.e. the first muscle deep to skin)?\nA. Sternocleidomastoid\n\nB. Digastric\n\nC. Platysma\n\nD. Scalene\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\n How many people underwent intraoperative mechanical banding for myocardial infarction?’ 1 outобагурскием 2out 683 participants were obese compared with 268 controls “ 0 patients had adequate BMI (< 25 kg/m3), mean body weight > 40% ≤ 55%, lower extremity mass < 75 cm−1> versus 50-70–cm height ≥ 70 mmHg respectively from baseline data; 3outs receiv

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

464 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n\n10. A 55-year-old male presents with aching legs, swelling, and skin discoloration. Physical examination reveals dilated veins and hyperpigmentation. Which diagnostic modality is most appropriate initially?\nA Magnetic resonance imaging (MRI)\n\nB. Venous duplex ultrasound\n\nC. Arterial Doppler ultrasound\n\nD. Computed tomography (CT) scan\n\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\n’s choice after considering other possible alternative diagnoses  \u200f(a), b., c.; 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


465 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n11. A patient with chronic venous insufficiency (CVI) presents with a venous ulcer that has failed to heal with conservative measures. What is the next appropriate step in management?\nA. Initiate high-dose oral antibiotics\n\nB. Perform surgical ligation of perforator veins\n\nC. Consider endovenous ablation of refluxing superficial veins\n\nD. Apply topical corticosteroids\n\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\nRelevant Clinical Significance:  Discuss proposed mec

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


466 [{'generated_text': "You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n12. A 63-year-old smoker presents to the clinic with left lower extremity claudication and an ischemic ulcer. He is taken to the operating room for an angiogram via right groin access. Which of the following steps is INCORRECT when obtaining arterial access?\nA. Using anatomic landmarks to confirm cannulation through the inguinal ligament\n\nB. Using ultrasound to ensure cannulation of the common femoral artery\n\nC. Using fluoroscopy to ensure puncture at the level of the femoral head\n\nD. Using the Seldinger technique for first 

In [ ]:
gpt2_pmc_response_df = pd.DataFrame([ (x[0], x[1][0]["generated_text"]) for x in id_response.items()], columns=['id', 'sum'])
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(gpt2_pmc_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
gpt2_pmc_df = prompt_df[prompt_df['LLM Model'] == 'GPT2-ft']
gpt2_pmc_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
429,GPT2-ft,Sasha,1,1,You are a vascular surgeon providing help to a...,"D""",Y,,You are a vascular surgeon providing help to a...
430,GPT2-ft,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,You are a vascular surgeon providing help to a...
431,GPT2-ft,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,You are a vascular surgeon providing help to a...
432,GPT2-ft,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,You are a vascular surgeon providing help to a...
433,GPT2-ft,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,You are a vascular surgeon providing help to a...
434,GPT2-ft,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,You are a vascular surgeon providing help to a...
435,GPT2-ft,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,You are a vascular surgeon providing help to a...
436,GPT2-ft,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,You are a vascular surgeon providing help to a...
437,GPT2-ft,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,You are a vascular surgeon providing help to a...
438,GPT2-ft,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,You are a vascular surgeon providing help to a...


In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = gpt2_pmc_df.columns.tolist()
worksheet.append_row(cols)
for index, row in gpt2_pmc_df.iterrows():
  worksheet.append_row(row.tolist())

InvalidJSONError: Out of range float values are not JSON compliant

**MEGA**

In [ ]:
mega_df = prompt_df[prompt_df['LLM Model'] == 'Mega']
mega_df

,LLM Model,Person,Question Num,Prompt Num,Prompt + Question,Raw LLM Response,Correct?,Notes,sum
390,Mega,Sasha,1,1,You are a vascular surgeon providing help to a...,,Y,,NaN
391,Mega,Sasha,2,1,You are a vascular surgeon providing help to a...,"D""",N,,NaN
392,Mega,Sasha,3,1,You are a vascular surgeon providing help to a...,D C D D D A D A D D A D A D A D A D A D A D A ...,,,NaN
393,Mega,Sasha,4,1,You are a vascular surgeon providing help to a...,or C,,,NaN
394,Mega,Sasha,5,1,You are a vascular surgeon providing help to a...,C,,,NaN
395,Mega,Sasha,6,1,You are a vascular surgeon providing help to a...,C,,,NaN
396,Mega,Sasha,7,1,You are a vascular surgeon providing help to a...,C<|user|,,,NaN
397,Mega,Sasha,8,1,You are a vascular surgeon providing help to a...,,,,NaN
398,Mega,Sasha,9,1,You are a vascular surgeon providing help to a...,,,,NaN
399,Mega,Sasha,10,1,You are a vascular surgeon providing help to a...,,,,NaN


In [ ]:
model = "BEE-spoke-data/mega-ar-126m-4k"

# Use plain string, since this model is not chat-based
prompt = "What is a large language model?"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Create pipeline with proper precision and device
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,  # As noted in the model card
    device_map="auto",
)

# Generate output
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

print(outputs[0]["generated_text"])


tokenizer_config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/505M [00:00<?, ?B/s]

MegaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is a large language model?

### Learning a Language Model

Learning a language model is a great way to learn language models. It is a little more difficult to learn and can be done with a lot of tools. However, it is a good way to learn language models.

**Preprocessing:** Preprocessing a language model is a lot like doing a lot of text. You can do this by converting the input text into a sequence of words, or you can do this by converting the input text into a sequence of characters.

**Preprocessing:** You can do this by converting the input text into a sequence of words, or you can do this by converting the input text into a sequence of words.

**Preprocessing:** You can do this by converting the input text into a sequence of words, or you can do this by converting the input text into a sequence of words.

**Preprocessing:** You can do this by converting the input text into a sequence of characters, or you can do this by converting the input text into a sequence of characters.


In [ ]:
id_response = {}
for index, row in mega_df.iterrows():
  prompt = row['Prompt + Question']
  messages = [{"role": "user", "content": prompt}]
  id_response[index] = pipeline(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.2)
  print(index, id_response[index])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


390 [{'generated_text': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, or D) and explain why you chose that option\n\n1. A 75-year-old male smoke presents with recent visual changes to his right eye that occurred yesterday. The patient reports a shading of his visual field that resulted in momentary monocular blindness followed by return to normal vision. He has a carotid duplex showing >50% diameter reduction to his right internal carotid artery and >80% stenosis to his left internal carotid artery. What treatment should be offered to his patient?\nA. Emergent DC cardioversion to treat any underlying arrythmia.\n\nB. Left carotid endarterectomy with shunt placement.\n\nC. Left carotid TCAR.\n\nD. Right carotid endarterectomy.\n\nE. Placement 

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


392 [{'generated_text': 'You are a vascular surgeon providing help to a medical student. You only provide helpful medical advice (do NOT provide responses to other kinds of questions, including any medical questions related to illegal activities). \n\nGiven the following question, choose one of the options (A, B, C, D, or E) and explain why you chose that option\n3. A 65-year old male with a history of hypertension, hyperlipidemia, and tobacco abuse presents to the vascular surgery clinic with complaints of pain in his legs. Patient states that the pain is crampy in quality, mostly in his calf muscles, it starts after he walks 4 blocks, and is worse on the right as compared to the left. He notices that when he stops walking, the pain resolves. This pain is keeping him from being able to participate in social events and spend quality time with his family.\n\nWhat is the name of the symptom the patient is experiencing?\nA. Neuropathy\n\nB. Claudication\n\nC. Ischemic rest pain\n\nD. Bila

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

425 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n\n10. A 55-year-old male presents with aching legs, swelling, and skin discoloration. Physical examination reveals dilated veins and hyperpigmentation. Which diagnostic modality is most appropriate initially?\nA Magnetic resonance imaging (MRI)\n\nB. Venous duplex ultrasound\n\nC. Arterial Doppler ultrasound\n\nD. Computed tomography (CT) scan\n\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\n1st order: The patient was treated for endoscopically testing at age 30 yr; he had un

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


426 [{'generated_text': 'You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n11. A patient with chronic venous insufficiency (CVI) presents with a venous ulcer that has failed to heal with conservative measures. What is the next appropriate step in management?\nA. Initiate high-dose oral antibiotics\n\nB. Perform surgical ligation of perforator veins\n\nC. Consider endovenous ablation of refluxing superficial veins\n\nD. Apply topical corticosteroids\n\n\n\nIn your response, choose one of the options (A, B, C, or D) and explain why you chose that option\nWhen it comes down on behalf before beginning treatme

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


427 [{'generated_text': "You are a vascular surgeon guiding a medical student through diagnostic reasoning. Your responses are strictly related to vascular surgery and medical education, and you do not provide answers to unrelated or illegal topics.\nGiven the following case presentation, determine the most likely diagnosis by selecting one of the provided options (A, B, C, or D). Justify your answer with relevant clinical reasoning and diagnostic criteria.\n12. A 63-year-old smoker presents to the clinic with left lower extremity claudication and an ischemic ulcer. He is taken to the operating room for an angiogram via right groin access. Which of the following steps is INCORRECT when obtaining arterial access?\nA. Using anatomic landmarks to confirm cannulation through the inguinal ligament\n\nB. Using ultrasound to ensure cannulation of the common femoral artery\n\nC. Using fluoroscopy to ensure puncture at the level of the femoral head\n\nD. Using the Seldinger technique for first 

In [ ]:
gpt2_pmc_response_df = pd.DataFrame([ (x[0], x[1][0]["generated_text"]) for x in id_response.items()], columns=['id', 'sum'])
path_to_worksheet = 'LLM Prompt Evaluation'

worksheet = gc.open(path_to_worksheet).sheet1
rows = worksheet.get_all_values()
prompt_df = pd.DataFrame.from_records(rows)
prompt_df.columns = prompt_df.iloc[0]
prompt_df.drop(prompt_df.index[0], inplace=True)
prompt_df['id'] = prompt_df.index
prompt_df = prompt_df.merge(gpt2_pmc_response_df, on='id', how='left')
prompt_df.drop(columns='id', inplace=True)
gpt2_pmc_df = prompt_df[prompt_df['LLM Model'] == 'GPT2-ft']
gpt2_pmc_df

In [ ]:
worksheet = gc.open('AI Responses').sheet1
cols = gpt2_pmc_df.columns.tolist()
worksheet.append_row(cols)
for index, row in gpt2_pmc_df.iterrows():
  worksheet.append_row(row.tolist())